# 4.4 - Autoencoders

En las próximas prácticas vamos a crear arquitecturas propias del aprendizaje no supervisado. Como ya sabrás, este tipo de problemas se caracterizan por la falta de etiquetas en los datos, lo que significa que no contamos con una respuesta correcta para cada entrada.

En su lugar, buscamos descubrir patrones, estructuras y relaciones dentro de los datos. Una de las redes neuronales profundas más utilizadas para este tipo de problemas son los **Autoencoders**.

Un Autoencoder es un tipo de red neuronal utilizada para aprender representaciones eficientes (codificaciones) de los datos, típicamente para la reducción de dimensionalidad.

Durante su entrenamiento tienen como objetivo reconstruir su entrada en la salida, pasando la información a través de una red de capas más pequeñas.

### Estructura de un Autoencoder

![image](https://i.imgur.com/0gIK8Gd.png)

Estos constan de dos partes principales:

1. **Codificador (Encoder)**: Comprime la entrada a un espacio *latente* de menor dimensión que el espacio original.
2. **Decodificador (Decoder)**: Reconstruye los datos originales a partir de la representación compacta creada por el codificador.

En esta práctica entrenaremos un Autoencoder convolucional capaz de aprender una codificación en un espacio de 2D ($\mathbb{R}^{2}$) para cada una de las imágenes de un conjunto.


## 4.4.1 - Conjunto de datos

Utilizaremos el conjunto [MNIST](https://es.wikipedia.org/wiki/Base_de_datos_MNIST) ya utilizado en prácticas anteriores. Como recordarás, este conjunto posee imágenes en escala de grises con números del 0 al 9 escritos a mano por personas. Cada una de estas imágenes tiene una resolución de $28 \times 28$ píxels. Al ser en escala de grises, cada imagen será un tensor de $1 \times 28 \times 28$.

Este conjunto de datos está pensado para resolver un problema de *Aprendizaje supervisado de multiclasificación*, pero en este caso descartaremos las etiquetas y utilizaremos solamente las imágenes.

### Descargar conjunto

Utilizaremos de nuevo  ``torchvision`` y su submódulo ``datasets`` para obtener el conjunto.


In [ ]:
import torch
from torch.utils.data import random_split
from torchvision import datasets, transforms

# Fijar la semilla para obtener reproducibilidad y crear variable device
seed = 42
torch.manual_seed(seed)  # Fijar semilla de PyTorch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Definimos una transformación que convierte las imágenes a tensores.
# La transformación ToTensor() convierte una imagen PIL en un tensor de PyTorch.
# El compose permite concatenar múltiples transformaciones, en este caso solo aplicamos una.
transform = transforms.Compose([transforms.ToTensor()])

# Descargamos el dataset indicando donde almacenarlo, la partición y las transformaciones
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Creamos un conjunto de validación, dado que no viene por defecto
mnist_train, mnist_val = random_split(mnist_train, (0.8, 0.2))

batch_size = 16

# Tras descargar, ya tentemos un objeto Dataset, por lo que necesitamos un DataLoader
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=False)
val_loader = torch.utils.data.DataLoader(mnist_val, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

## 4.4.2 - Autoencoder Convolucional

Los autoencoders no poseen una arquitectura definida; esta depende del problema y del tipo de datos con los que trabajemos. Lo único que definen es la necesidad de crear una parte *encoder* y otra *decoder*.

Al trabajar con imágenes, nuestro encoder estará formado por capas convolucionales que procesarán la imagen y la proyectarán en un espacio de, en este caso, $\mathbb{R}^{2}$. 

El decoder hará el proceso inverso: deberá generar, dado un vector 2D, una salida de $1 \times 28 \times 28$. Para llevar a cabo este proceso serán necesarias capas denominadas *deconvoluciones* o *convoluciones transpuestas*.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Definir el autoencoder convolucional
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2, padding=1),  # (batch, 8, 14, 14)
            nn.Tanh(),
            nn.Conv2d(8, 16, 3, stride=2, padding=1),  # (batch, 16, 7, 7)
            nn.Tanh(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # (batch, 32, 4, 4)
            nn.Tanh(),
            nn.Flatten(),
            nn.Linear(32 * 4 * 4, 64),  # Proyección a vector de 64D
            nn.Tanh(),
            nn.Linear(64, 2)  # Proyección a vector de 2D
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(2, 64),
            nn.Tanh(),
            nn.Linear(64, 32 * 4 * 4),
            nn.Tanh(),
            nn.Unflatten(1, (32, 4, 4)),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1),  # (batch, 16, 7, 7)
            nn.Tanh(),
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),  # (batch, 8, 14, 14)
            nn.Tanh(),
            nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1),  # (batch, 1, 28, 28)
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# Configuración de parámetros y carga del dataset MNIST
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Inicialización del modelo, criterio de pérdida y optimizador
model = ConvAutoencoder()
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

A continuación entrenamos el modelo pasando como entrada y salida el mismo elemento.

In [ ]:
import matplotlib.pyplot as plt

# Entrenamiento del autoencoder
for epoch in range(num_epochs):
    model.train()
    for data in train_loader:
        img, _ = data # Como se puede ver, no necesitamos las etiquetas en este caso
        img = img.to(device)
        output = model(img)
        loss = criterion(output, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validación
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            img, _ = data # Como se puede ver, no necesitamos las etiquetas en este caso
            img = img.to(device)
            output = model(img)
            val_loss += criterion(output, img).item()
    
    val_loss /= len(val_loader)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f} Validation Loss: {val_loss:.4f}')
    
    # Mostrar una imagen del conjunto de validación y su predicción
    val_data_iter = iter(val_loader) 
    val_img, _ = next(val_data_iter)  # Obtener una imagen del conjunto de validación
    val_img = val_img.to(device)
    with torch.no_grad():
        val_output = model(val_img)

    # Visualización
    plt.figure(figsize=(9, 3))
    
    # Imagen original
    plt.subplot(1, 2, 1)
    plt.title("Imagen Original")
    plt.imshow(val_img[0].cpu().squeeze(), cmap='gray')
    plt.axis('off')

    # Imagen reconstruida
    plt.subplot(1, 2, 2)
    plt.title("Predicción del Modelo")
    plt.imshow(val_output[0].cpu().squeeze(), cmap='gray')
    plt.axis('off')

    plt.show()

## 4.4.3. - Ejercicios

> **EJERCICIO:** Modifica y reentrena el autoencoder para que proyecte a un vector en $\mathbb{R}^{8}$ en vez de $\mathbb{R}^{2}$. A continuación crea un modelo para resolver el problema de multi-clasficación MNIST utilizando como entrada, en vez de las imágenes, el vector aprendido.

> **EJERCICIO:** Compara la accuracy en test del modelo de multi-clasificación anterior con uno que utilice las imágenes como entrada.